# Lesson 5: Reranking Retrieved Documents

**Objective**: Enhance the relevance of retrieved documents to improve the quality of generated responses.

**Topics**:
- Reranking techniques
- Trade-offs between speed and accuracy

**Practical Task**: Implement a reranking model and evaluate its impact on retrieval performance.

**Resources**:
- Cohere reranker
- Open-source alternative


In [52]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_community.vectorstores import Qdrant
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

load_dotenv()

file_path = (
    "../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
legal_docs = loader.load_and_split()

for doc in legal_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')


In [53]:
from langchain_ollama import OllamaLLM
#!ollama pull llama3.1
#llm = OllamaLLM(model="llama3.2")
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [54]:
legal_docs[0].page_content.replace('\n', ' ')

'Status:  This is the original version (as it was originally made). STATUTORY INSTRUMENTS 2003 No. 1659 FOOD, ENGLAND The Cocoa and Chocolate Products (England) Regulations 2003 Made       -      -      -      - 25th June 2003 Laid before Parliament 3rd July 2003 Coming into force       -      - 3rd August 2003 The Secretary of State, in exercise of the powers conferred by sections 16(1)(e), 17(1), 26(1) and (3) and 48(1) of the Food Safety Act 1990(1) and now vested in him(2) and of all other powers enabling him in that behalf, having had regard in accordance with section 48(4A) of that Act to relevant advice given by the Food Standards Agency, and after consultation both as required by Article 9 of Regulation (EC) No. 178/2002 of the European Parliament and of the Council laying down the general principles and requirements of food law, establishing the European Food Safety Authority and laying down procedures in matters of food safety (3) and in accordance with section 48(4) and (4B)

In [55]:
legal_docs

[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 0}, page_content='Status:  This is the original version (as it was originally made). STATUTORY INSTRUMENTS 2003 No. 1659 FOOD, ENGLAND The Cocoa and Chocolate Products (England) Regulations 2003 Made       -      -      -      - 25th June 2003 Laid before Parliament 3rd July 2003 Coming into force       -      - 3rd August 2003 The Secretary of State, in exercise of the powers conferred by sections 16(1)(e), 17(1), 26(1) and (3) and 48(1) of the Food Safety Act 1990(1) and now vested in him(2) and of all other powers enabling him in that behalf, having had regard in accordance with section 48(4A) of that Act to relevant advice given by the Food Standards Agency, and after consultation both as required by Article 9 of Regulation (EC) No. 178/2002 of the European Parliament and of the Council laying down the general principles and requirements of food law, establishing the European Food

In [56]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
        },
    ),
]
vectorstore = Qdrant.from_documents(
    docs,
    embedding_model,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

## Self-query retriever

In [15]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.self_query.qdrant import QdrantTranslator

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"

structured_query_translator = QdrantTranslator(metadata_key="metadata")

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True,
    structured_query_translator=structured_query_translator,
)

In [16]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")

[Document(metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', '_id': '0ad7a13cab5c400585586e27bb0ec5b8', '_collection_name': 'my_documents'}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(metadata={'year': 1995, 'genre': 'animated', '_id': '6ba6a38cf2114de6a2ef62efe7cfffcd', '_collection_name': 'my_documents'}, page_content='Toys come alive and have a blast doing so'),
 Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', '_id': '7eff7d2c1d314fee8d03dc92c25cad6e', '_collection_name': 'my_documents'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(metadata={'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2, '_id': '2d4238f31e9b423bbcf1ff5e995998c6', '_collection_name': 'my_documents'}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...')]

In [17]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', '_id': '7eff7d2c1d314fee8d03dc92c25cad6e', '_collection_name': 'my_documents'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, '_id': '135f470a2aed4fc3b85b4b64e5454361', '_collection_name': 'my_documents'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [18]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3, '_id': '59d0c3df1f9541b6b13e0f8c26e281dc', '_collection_name': 'my_documents'}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them')]

In [19]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, '_id': '135f470a2aed4fc3b85b4b64e5454361', '_collection_name': 'my_documents'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', '_id': '7eff7d2c1d314fee8d03dc92c25cad6e', '_collection_name': 'my_documents'}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

### Parent Document Retrieval

In [20]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [21]:
file_path = (
    "../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load()

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [23]:
# Create both stores
store = InMemoryStore()  # For storing the parent documents
byte_store = InMemoryStore()  # For storing the child document chunks

parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,  # Changed from store to docstore
    byte_store=byte_store,  # Added byte_store
    parent_splitter=parent_splitter,
    child_splitter=child_splitter
)


In [24]:
len(docs)

13

In [25]:
parent_retriever.add_documents(docs)

In [26]:
len(list(byte_store.yield_keys()))

43

In [27]:
sub_docs = vectorstore.similarity_search("What is chocolate?")
sub_docs

[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 6, 'doc_id': 'f1e4802a-ff12-49cc-a089-036ac3185f93', '_id': '8eb50918ae6f4f33bf3a1156876727be', '_collection_name': 'my_documents'}, page_content='(c) (c)  Powdered chocolate or\nChocolate in powder\nThe product consisting of a mixture of cocoa\npowder and sugars, containing not less than 32\nper cent cocoa powder.\n(d) (d)  Drinking chocolate or'),
 Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 7, 'doc_id': '8b93dcf8-9f0f-4472-adaa-d2f9ac3658d1', '_id': 'cd735e6e1c5c4f77954d5c9f6cf5fcb8', '_collection_name': 'my_documents'}, page_content='(a) (a)  Milk chocolate (a) The product obtained from cocoa\nproducts, sugars and milk or milk\nproducts which, subject to item 4(b),\ncontains—\n— not less than 25 per cent total dry\ncocoa solids;'),
 Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf'

In [28]:
print(len(sub_docs[0].page_content))


196


In [29]:
retrieved_docs = parent_retriever.invoke("What is chocolate?")
print(len(retrieved_docs[0].page_content))

997


### Hybrid Search

#### Dense Retrieval

In [30]:
from langchain_qdrant import RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    legal_docs,
    embedding=embedding_model,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

query = "What is chocolate?"
found_docs = qdrant.similarity_search(query)

In [31]:
found_docs

[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 9, '_id': '89c52160d164421d85a707fc580022d9', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n— not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6. White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7. Filled chocolate or\nChocolate with … filling or\nChocolate with … centre\nThe filled product, the outer part of which\nconsists of a product specified in column\

#### Sparse Retrieval

In [32]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

qdrant = QdrantVectorStore.from_documents(
    legal_docs,
    embedding=embedding_model,
    sparse_embedding=sparse_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.SPARSE,
)

query = "What is chocolate?"
found_docs = qdrant.similarity_search(query)
found_docs

Fetching 29 files: 100%|██████████| 29/29 [00:01<00:00, 14.95it/s]


[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 11, '_id': 'aea89a744d94465bb44912f6ae18fb7a', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nUsual name of vegetable fat Scientific name of the plants from which\nthe fats listed can be obtained\n6. Mango kernel Mangifera indica\n3. Coconut oil may be used in chocolate for the manufacture of ice cream and similar frozen\nproducts.\n4. In this Schedule—\n“P” means palmitic acid;\n“O” means oleic acid;\n“St” means stearic acid.\nEXPLANATORY NOTE\n(This note is not part of the Regulations)\nThese Regulations, which apply to England, implement Directive 2000/36/EC of the European\nParliament and the Council relating to cocoa and chocolate products intended for human\nconsumption(23). They revoke and replace the Cocoa and Chocolate Products Regulations 1976, as\namended, in relation to 

#### Hybrid Retrieval

In [33]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

qdrant = QdrantVectorStore.from_documents(
    legal_docs,
    embedding=embedding_model,
    sparse_embedding=sparse_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.HYBRID,
)

query = "What is chocolate?"
found_docs = qdrant.similarity_search(query)
found_docs

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 318415.75it/s]


[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 9, '_id': '9aedf4fcadb244b6bff5ab38291961c8', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n— not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6. White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7. Filled chocolate or\nChocolate with … filling or\nChocolate with … centre\nThe filled product, the outer part of which\nconsists of a product specified in column\

### Query Rewriting

In [34]:
from langchain_core.prompts import ChatPromptTemplate
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [35]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")

/Users/josecarlos/repos/practicos-rag/.venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
print(rewrite_prompt.template)

Provide a better search query for web search engine to answer the given question, end the queries with ’**’.  Question {x} Answer:


In [45]:
from langchain_qdrant import RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embedding_model,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

query = "What is chocolate?"
found_docs = qdrant.similarity_search(query)

In [46]:
found_docs

[Document(metadata={'source': '../practicos-rag/data/Regulaciones cacao y chocolate 2003.pdf', 'page': 9, '_id': '9b08b82a650a4a80a3453afbf7202845', '_collection_name': 'my_documents'}, page_content='Document Generated: 2023-04-24\nStatus:  This is the original version (as it was originally made).\nColumn 1 Column 2\nReserved Descriptions Designated Products\n— not less than 25 per cent total fat (cocoa\nbutter and milk fat).\n6. White chocolate The product obtained from cocoa butter, milk\nor milk products and sugars which contains not\nless than 20 per cent cocoa butter and not less\nthan 14 per cent dry milk solids obtained by\npartly or wholly dehydrating whole milk, semi-\nskimmed or skimmed milk, cream, or from\npartly or wholly dehydrated cream, butter or\nmilk fat, of which not less than 3.5 per cent is\nmilk fat.\n7. Filled chocolate or\nChocolate with … filling or\nChocolate with … centre\nThe filled product, the outer part of which\nconsists of a product specified in column\

### Hypothetical Document Embedding

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert about a set of software for building LLM-powered applications called LangChain, LangGraph, LangServe, and LangSmith.

LangChain is a Python framework that provides a large set of integrations that can easily be composed to build LLM applications.
LangGraph is a Python package built on top of LangChain that makes it easy to build stateful, multi-actor LLM applications.
LangServe is a Python package built on top of LangChain that makes it easy to deploy a LangChain application as a REST API.
LangSmith is a platform that makes it easy to trace and test LLM applications.

Answer the user question as best you can. Answer as though you were writing a tutorial that addressed the user question."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
qa_no_context = prompt | llm | StrOutputParser()

In [41]:
answer = qa_no_context.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)
print(answer)

To use multi-modal models in a chain and turn that chain into a REST API, you can leverage the capabilities of LangChain and LangServe. Below is a step-by-step tutorial that will guide you through the process.

### Step 1: Set Up Your Environment

First, ensure you have Python installed on your machine. Then, install the necessary packages:

```bash
pip install langchain langserve
```

### Step 2: Create a Multi-Modal Chain

In this example, we will create a simple multi-modal chain that can handle both text and image inputs. For demonstration purposes, let's assume we are using a hypothetical multi-modal model that can process both types of data.

```python
from langchain import Chain
from langchain.models import MultiModalModel

# Initialize your multi-modal model
model = MultiModalModel()

# Create a chain that processes both text and image inputs
class MultiModalChain(Chain):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def run(self, tex

In [42]:
from langchain_core.runnables import RunnablePassthrough

hyde_chain = RunnablePassthrough.assign(hypothetical_document=qa_no_context)

hyde_chain.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)

{'question': 'how to use multi-modal models in a chain and turn chain into a rest api',
 'hypothetical_document': 'To use multi-modal models in a chain and turn that chain into a REST API, you can leverage the capabilities of LangChain and LangServe. Below is a step-by-step tutorial that guides you through the process.\n\n### Step 1: Set Up Your Environment\n\nFirst, ensure you have Python installed on your machine. Then, install the necessary packages:\n\n```bash\npip install langchain langserve\n```\n\n### Step 2: Create a Multi-Modal Chain\n\nMulti-modal models can process different types of data, such as text, images, and audio. In this example, we will create a simple chain that takes both text and image inputs.\n\n```python\nfrom langchain import Chain\nfrom langchain.prompts import PromptTemplate\nfrom langchain.llms import OpenAI\nfrom langchain.chains import SimpleSequentialChain\n\n# Define your multi-modal model (for example, OpenAI\'s CLIP model for images and text)\ntext_m

### Reranking

In [43]:
from dotenv import load_dotenv

load_dotenv()

True

In [44]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [47]:
pretty_print_docs(found_docs)

Document 1:

Document Generated: 2023-04-24
Status:  This is the original version (as it was originally made).
Column 1 Column 2
Reserved Descriptions Designated Products
— not less than 25 per cent total fat (cocoa
butter and milk fat).
6. White chocolate The product obtained from cocoa butter, milk
or milk products and sugars which contains not
less than 20 per cent cocoa butter and not less
than 14 per cent dry milk solids obtained by
partly or wholly dehydrating whole milk, semi-
skimmed or skimmed milk, cream, or from
partly or wholly dehydrated cream, butter or
milk fat, of which not less than 3.5 per cent is
milk fat.
7. Filled chocolate or
Chocolate with … filling or
Chocolate with … centre
The filled product, the outer part of which
consists of a product specified in column
2 of item 3, 4, 5 or 6 of this Schedule and
constitutes not less than 25 per cent of the total
weight of the product, but does not include any
filled product, the inside of which consists of
bakery products

In [58]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

llm = Cohere(temperature=0)
compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

/var/folders/p9/dtzw1bk549g0ft33yw9bnbpc0000gq/T/ipykernel_1661/2282730921.py:5: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(temperature=0)


Document 1:

Signed by authority of the Secretary of State for Health
25th June 2003
Melanie Johnson
Parliamentary Under-Secretary of State,
Department of Health
(17) S.I. 1982/17.
----------------------------------------------------------------------------------------------------
Document 2:

(19) OJ No. L197, 3.8.2000, p.19.
(20) S.I. 1995/3124 to which there are amendments not relevant to these Regulations.
(21) S.I. 1995/3187; the relevant amending instrument is S.I. 1999/1136.
6
----------------------------------------------------------------------------------------------------
Document 3:

(4) OJ No. L1, 3.1.94, p.1.
(5) OJ No. L1, 3.1.94, p.571.
(6) S.I. 1996/1499, the relevant amending instrument is S.I. 1998/1398.
2
